In [1]:
#Importamos la librería pandas que necesitamos para la lectura, conversión y limpieza de los datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np
import re 

#Importamos librerías necesarias para la visualización
# -----------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Imputación de nulos usando métodos avanzados estadísticos
# -----------------------------------------------------------------------
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

#Importamos librerías para localizar ruta archivos.
import os
import sys
sys.path.append("../")


pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames
# pd.set_option('display.max_rows', None) # para poder visualizar todas las filas de los DataFrames

In [2]:
#Abrimos el csv y lo convertimos a un DataFrame.
df = pd.read_csv('finanzas-hotel-bookings.csv', index_col= 0)
df.head(10)

/Users/reyesaltozano/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (1,2,5,13,14,15,16,20,21,26,30,32) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,0
0,Resort Hotel,False,342.0,2015.0,July,27.0,1.0,0.0,0.0,2.0,NaN,0.0,BB,PRT,NaN,Direct,0.0,NaN,0.0,C,C,3.0,NaN,NaN,0.0,Transient,0.0,0.0,0.0,Check-Out,2015-07-01 00:00:00,NaN
1,Resort Hotel,False,737.0,NaN,July,27.0,1.0,0.0,0.0,2.0,NaN,0.0,BB,NaN,NaN,Direct,0.0,0.0,0.0,NaN,C,4.0,NaN,NaN,0.0,Transient,0.0,0.0,0.0,Check-Out,2015-07-01 00:00:00,NaN
2,Resort Hotel,False,7.0,2015.0,July,27.0,1.0,0.0,1.0,1.0,0.0,0.0,BB,GBR,NaN,Direct,0.0,0.0,0.0,A,C,0.0,NaN,NaN,0.0,Transient,75.0,0.0,0.0,Check-Out,2015-07-02 00:00:00,NaN
3,Resort Hotel,False,13.0,NaN,July,27.0,1.0,0.0,1.0,1.0,NaN,0.0,BB,GBR,Corporate,Corporate,0.0,0.0,0.0,A,A,0.0,304.0,NaN,0.0,Transient,75.0,0.0,0.0,Check-Out,2015-07-02 00:00:00,NaN
4,Resort Hotel,False,14.0,NaN,July,NaN,1.0,0.0,2.0,2.0,NaN,0.0,BB,NaN,Online TA,TA/TO,0.0,0.0,0.0,A,A,0.0,240.0,NaN,0.0,Transient,98.0,0.0,1.0,Check-Out,2015-07-03 00:00:00,NaN
5,Resort Hotel,False,14.0,NaN,July,27.0,1.0,0.0,2.0,2.0,NaN,0.0,BB,GBR,Online TA,TA/TO,0.0,0.0,0.0,A,A,0.0,240.0,NaN,0.0,NaN,98.0,0.0,1.0,Check-Out,2015-07-03 00:00:00,NaN
6,Resort Hotel,False,0.0,2015.0,July,NaN,1.0,0.0,2.0,2.0,NaN,0.0,BB,NaN,Direct,Direct,0.0,NaN,0.0,C,C,0.0,NaN,NaN,0.0,NaN,107.0,0.0,0.0,Check-Out,2015-07-03 00:00:00,NaN
7,Resort Hotel,False,9.0,2015.0,July,27.0,1.0,0.0,2.0,2.0,0.0,0.0,FB,NaN,Direct,Direct,0.0,NaN,0.0,C,C,0.0,303.0,NaN,0.0,NaN,103.0,0.0,1.0,Check-Out,2015-07-03 00:00:00,NaN
8,Resort Hotel,True,85.0,2015.0,July,27.0,1.0,0.0,3.0,2.0,0.0,0.0,BB,PRT,NaN,NaN,0.0,NaN,0.0,A,A,0.0,240.0,NaN,0.0,Transient,82.0,0.0,1.0,Canceled,2015-05-06 00:00:00,NaN
9,Resort Hotel,True,75.0,2015.0,July,27.0,1.0,0.0,3.0,2.0,0.0,0.0,HB,NaN,NaN,TA/TO,0.0,0.0,0.0,D,D,0.0,15.0,NaN,0.0,Transient,105.5,0.0,0.0,Canceled,2015-04-22 00:00:00,NaN


In [3]:
# COLUMNA IS_CANCELED: Cambiamos los valores True y False a Yes y No

df['is_canceled'] = df['is_canceled'].map({False: 'No', True: 'Yes'})
df.head(10)

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,0
0,Resort Hotel,No,342.0,2015.0,July,27.0,1.0,0.0,0.0,2.0,NaN,0.0,BB,PRT,NaN,Direct,0.0,NaN,0.0,C,C,3.0,NaN,NaN,0.0,Transient,0.0,0.0,0.0,Check-Out,2015-07-01 00:00:00,NaN
1,Resort Hotel,No,737.0,NaN,July,27.0,1.0,0.0,0.0,2.0,NaN,0.0,BB,NaN,NaN,Direct,0.0,0.0,0.0,NaN,C,4.0,NaN,NaN,0.0,Transient,0.0,0.0,0.0,Check-Out,2015-07-01 00:00:00,NaN
2,Resort Hotel,No,7.0,2015.0,July,27.0,1.0,0.0,1.0,1.0,0.0,0.0,BB,GBR,NaN,Direct,0.0,0.0,0.0,A,C,0.0,NaN,NaN,0.0,Transient,75.0,0.0,0.0,Check-Out,2015-07-02 00:00:00,NaN
3,Resort Hotel,No,13.0,NaN,July,27.0,1.0,0.0,1.0,1.0,NaN,0.0,BB,GBR,Corporate,Corporate,0.0,0.0,0.0,A,A,0.0,304.0,NaN,0.0,Transient,75.0,0.0,0.0,Check-Out,2015-07-02 00:00:00,NaN
4,Resort Hotel,No,14.0,NaN,July,NaN,1.0,0.0,2.0,2.0,NaN,0.0,BB,NaN,Online TA,TA/TO,0.0,0.0,0.0,A,A,0.0,240.0,NaN,0.0,Transient,98.0,0.0,1.0,Check-Out,2015-07-03 00:00:00,NaN
5,Resort Hotel,No,14.0,NaN,July,27.0,1.0,0.0,2.0,2.0,NaN,0.0,BB,GBR,Online TA,TA/TO,0.0,0.0,0.0,A,A,0.0,240.0,NaN,0.0,NaN,98.0,0.0,1.0,Check-Out,2015-07-03 00:00:00,NaN
6,Resort Hotel,No,0.0,2015.0,July,NaN,1.0,0.0,2.0,2.0,NaN,0.0,BB,NaN,Direct,Direct,0.0,NaN,0.0,C,C,0.0,NaN,NaN,0.0,NaN,107.0,0.0,0.0,Check-Out,2015-07-03 00:00:00,NaN
7,Resort Hotel,No,9.0,2015.0,July,27.0,1.0,0.0,2.0,2.0,0.0,0.0,FB,NaN,Direct,Direct,0.0,NaN,0.0,C,C,0.0,303.0,NaN,0.0,NaN,103.0,0.0,1.0,Check-Out,2015-07-03 00:00:00,NaN
8,Resort Hotel,Yes,85.0,2015.0,July,27.0,1.0,0.0,3.0,2.0,0.0,0.0,BB,PRT,NaN,NaN,0.0,NaN,0.0,A,A,0.0,240.0,NaN,0.0,Transient,82.0,0.0,1.0,Canceled,2015-05-06 00:00:00,NaN
9,Resort Hotel,Yes,75.0,2015.0,July,27.0,1.0,0.0,3.0,2.0,0.0,0.0,HB,NaN,NaN,TA/TO,0.0,0.0,0.0,D,D,0.0,15.0,NaN,0.0,Transient,105.5,0.0,0.0,Canceled,2015-04-22 00:00:00,NaN


In [4]:
# COLUMNA ARRIVAL_DATE_MONTH: Pasamos los nombres de los meses a un valor númerico: January a 1, February a 2 etc. 

# Creamos un diccionario para mapear los nombres de los meses a números
month_mapping = {
    'January': 1, 'February': 2, 'March': 3, 'April': 4,
    'May': 5, 'June': 6, 'July': 7, 'August': 8,
    'September': 9, 'October': 10, 'November': 11, 'December': 12
}

# Cambiamos los valores de la columna 'arrival_date_month'
df['arrival_date_month'] = df['arrival_date_month'].map(month_mapping)


# Mostramos el dataframe
df.head()

# No podemos pasarlo a int hasta que no hayamos hecho algo con los valores NaN

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,0
0,Resort Hotel,No,342.0,2015.0,7.0,27.0,1.0,0.0,0.0,2.0,NaN,0.0,BB,PRT,NaN,Direct,0.0,NaN,0.0,C,C,3.0,NaN,NaN,0.0,Transient,0.0,0.0,0.0,Check-Out,2015-07-01 00:00:00,NaN
1,Resort Hotel,No,737.0,NaN,7.0,27.0,1.0,0.0,0.0,2.0,NaN,0.0,BB,NaN,NaN,Direct,0.0,0.0,0.0,NaN,C,4.0,NaN,NaN,0.0,Transient,0.0,0.0,0.0,Check-Out,2015-07-01 00:00:00,NaN
2,Resort Hotel,No,7.0,2015.0,7.0,27.0,1.0,0.0,1.0,1.0,0.0,0.0,BB,GBR,NaN,Direct,0.0,0.0,0.0,A,C,0.0,NaN,NaN,0.0,Transient,75.0,0.0,0.0,Check-Out,2015-07-02 00:00:00,NaN
3,Resort Hotel,No,13.0,NaN,7.0,27.0,1.0,0.0,1.0,1.0,NaN,0.0,BB,GBR,Corporate,Corporate,0.0,0.0,0.0,A,A,0.0,304.0,NaN,0.0,Transient,75.0,0.0,0.0,Check-Out,2015-07-02 00:00:00,NaN
4,Resort Hotel,No,14.0,NaN,7.0,NaN,1.0,0.0,2.0,2.0,NaN,0.0,BB,NaN,Online TA,TA/TO,0.0,0.0,0.0,A,A,0.0,240.0,NaN,0.0,Transient,98.0,0.0,1.0,Check-Out,2015-07-03 00:00:00,NaN
5,Resort Hotel,No,14.0,NaN,7.0,27.0,1.0,0.0,2.0,2.0,NaN,0.0,BB,GBR,Online TA,TA/TO,0.0,0.0,0.0,A,A,0.0,240.0,NaN,0.0,NaN,98.0,0.0,1.0,Check-Out,2015-07-03 00:00:00,NaN
6,Resort Hotel,No,0.0,2015.0,7.0,NaN,1.0,0.0,2.0,2.0,NaN,0.0,BB,NaN,Direct,Direct,0.0,NaN,0.0,C,C,0.0,NaN,NaN,0.0,NaN,107.0,0.0,0.0,Check-Out,2015-07-03 00:00:00,NaN
7,Resort Hotel,No,9.0,2015.0,7.0,27.0,1.0,0.0,2.0,2.0,0.0,0.0,FB,NaN,Direct,Direct,0.0,NaN,0.0,C,C,0.0,303.0,NaN,0.0,NaN,103.0,0.0,1.0,Check-Out,2015-07-03 00:00:00,NaN
8,Resort Hotel,Yes,85.0,2015.0,7.0,27.0,1.0,0.0,3.0,2.0,0.0,0.0,BB,PRT,NaN,NaN,0.0,NaN,0.0,A,A,0.0,240.0,NaN,0.0,Transient,82.0,0.0,1.0,Canceled,2015-05-06 00:00:00,NaN
9,Resort Hotel,Yes,75.0,2015.0,7.0,27.0,1.0,0.0,3.0,2.0,0.0,0.0,HB,NaN,NaN,TA/TO,0.0,0.0,0.0,D,D,0.0,15.0,NaN,0.0,Transient,105.5,0.0,0.0,Canceled,2015-04-22 00:00:00,NaN


In [17]:
df.describe(include = "object").T

,count,unique,top,freq
hotel,119390,2,City Hotel,79330
is_canceled,119390,2,No,75166
meal,119390,5,BB,92310
country,65054,163,PRT,26577
market_segment,59934,8,Online TA,28331
distribution_channel,105899,5,TA/TO,86822
reserved_room_type,79275,10,A,57202
assigned_room_type,119390,12,A,74053
customer_type,94199,4,Transient,70610
reservation_status,119390,3,Check-Out,75166


# IMPUTACIÓN DE NULOS EN COLUMNAS CATEGÓRICAS


In [6]:
# Obtenemos la lista de columnas categóricas que tienen nulos
nulos_cat = df[df.columns[df.isnull().any()]].select_dtypes(include = "O").columns
print("Las columnas categóricas que tienen nulos son : \n ")
print(nulos_cat)

Las columnas categóricas que tienen nulos son : 
 
Index(['hotel', 'is_canceled', 'meal', 'country', 'market_segment',
       'distribution_channel', 'reserved_room_type', 'assigned_room_type',
       'customer_type', 'reservation_status', 'reservation_status_date', '0'],
      dtype='object')


In [7]:
for columna in nulos_cat:
    # Calcular el número total de filas
    total_filas = len(df)
    
    # Calcular el número de valores nulos en la columna actual
    num_nulos = df[columna].isna().sum()
    
    # Calcular el porcentaje de valores nulos en la columna actual
    porcentaje_nulos = (num_nulos / total_filas) * 100
    
    # Calcular el número de ocurrencias de cada valor único en la columna actual
    cuenta_valores = df[columna].value_counts()
    
    # Calcular el porcentaje de cada valor único en la columna actual
    porcentaje_valores = (cuenta_valores / total_filas) * 100
    
    # Mostrar resultados
    print(f"\nColumna: {columna.upper()}")
    print(f"Porcentaje de valores nulos: {porcentaje_nulos:.2f}%")
    print("Porcentaje de cada valor único:")
    print(porcentaje_valores)


Columna: HOTEL
Porcentaje de valores nulos: 34.72%
Porcentaje de cada valor único:
City Hotel      43.378883
Resort Hotel    21.905434
Name: hotel, dtype: float64

Columna: IS_CANCELED
Porcentaje de valores nulos: 34.72%
Porcentaje de cada valor único:
No     41.101943
Yes    24.182374
Name: is_canceled, dtype: float64

Columna: MEAL
Porcentaje de valores nulos: 34.72%
Porcentaje de cada valor único:
BB           50.476550
HB            7.908594
SC            5.823586
Undefined     0.639227
FB            0.436359
Name: meal, dtype: float64

Columna: COUNTRY
Porcentaje de valores nulos: 64.43%
Porcentaje de cada valor único:
PRT    14.532719
GBR     3.600781
FRA     3.147471
ESP     2.576595
DEU     2.171952
         ...    
LIE     0.000547
NIC     0.000547
GGY     0.000547
FJI     0.000547
UMI     0.000547
Name: country, Length: 163, dtype: float64

Columna: MARKET_SEGMENT
Porcentaje de valores nulos: 67.23%
Porcentaje de cada valor único:
Online TA        15.491833
Offline TA/TO    

### OPCIONES PARA IMPUTAR VALORES NULOS EN COLUMNAS CATEGÓRICAS

1. Rellenar los valores nulos con la categoría más frecuente:
Esta es una técnica común donde reemplazamos los valores nulos con la categoría que más frecuentemente aparece en la columna.

2. Rellenar los valores nulos de acuerdo a la distribución de las categorías:
Si deseas mantener la distribución original de las categorías al reemplazar los nulos, puedes hacerlo de acuerdo a la proporción existente.

3. Eliminar las filas con valores nulos:
Si los valores nulos son significativos y no quieres hacer asunciones, puedes optar por eliminar las filas con valores nulos.



In [7]:
# FÓRMULA PARA IMPUTAR DE ACUERDO CON LA DISTRIBUCIÓN. PRUEBO CON LA COLUMNA Hotel


# Mostrar la distribución original
print("Distribución original de las categorías:")
print(df['hotel'].value_counts())

# Calcular el porcentaje de nulos
null_percentage = df['hotel'].isna().mean() * 100
print(f"\nPorcentaje de nulos: {null_percentage:.2f}%")

# Definir las proporciones de las categorías existentes
category_proportions = {
    'City Hotel': 0.433789,
    'Resort Hotel': 0.219054
}

# Aumentar la proporción para rellenar los nulos
total_proportion = sum(category_proportions.values())
missing_proportion = 1 - total_proportion
category_proportions['City Hotel'] += (missing_proportion * (category_proportions['City Hotel'] / total_proportion))
category_proportions['Resort Hotel'] += (missing_proportion * (category_proportions['Resort Hotel'] / total_proportion))

# Crear una lista de categorías basada en la distribución ajustada
categories = df['hotel'].dropna().tolist()
num_nans = df['hotel'].isna().sum()
fill_values = np.random.choice(['City Hotel', 'Resort Hotel'], size=num_nans, p=[category_proportions['City Hotel'], category_proportions['Resort Hotel']])

# Reemplazar los valores nulos con los valores generados
df.loc[df['hotel'].isna(), 'hotel'] = fill_values

# Mostrar la nueva distribución
print("\nNueva distribución de las categorías:")
df['hotel'].value_counts(normalize=True)

# Mostrar el dataframe resultante
print("\nDataFrame resultante:")
df

Distribución original de las categorías:
City Hotel      106870
Resort Hotel     76007
Name: hotel, dtype: int64

Porcentaje de nulos: 0.00%

Nueva distribución de las categorías:

DataFrame resultante:


,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,0
0,Resort Hotel,No,342.000000,2015.0,July,27.0,1.0,0.000000,0.000000,2.0,NaN,0.0,BB,PRT,NaN,Direct,0.0,NaN,0.0,C,C,3.0,NaN,NaN,0.0,Transient,0.0,0.0,0.0,Check-Out,2015-07-01 00:00:00,NaN
1,Resort Hotel,No,737.000000,NaN,July,27.0,1.0,0.000000,0.000000,2.0,NaN,0.0,BB,NaN,NaN,Direct,0.0,0.0,0.0,NaN,C,4.0,NaN,NaN,0.0,Transient,0.0,0.0,0.0,Check-Out,2015-07-01 00:00:00,NaN
2,Resort Hotel,No,7.000000,2015.0,July,27.0,1.0,0.000000,1.000000,1.0,0.0,0.0,BB,GBR,NaN,Direct,0.0,0.0,0.0,A,C,0.0,NaN,NaN,0.0,Transient,75.0,0.0,0.0,Check-Out,2015-07-02 00:00:00,NaN
3,Resort Hotel,No,13.000000,NaN,July,27.0,1.0,0.000000,1.000000,1.0,NaN,0.0,BB,GBR,Corporate,Corporate,0.0,0.0,0.0,A,A,0.0,304.0,NaN,0.0,Transient,75.0,0.0,0.0,Check-Out,2015-07-02 00:00:00,NaN
4,Resort Hotel,No,14.000000,NaN,July,NaN,1.0,0.000000,2.000000,2.0,NaN,0.0,BB,NaN,Online TA,TA/TO,0.0,0.0,0.0,A,A,0.0,240.0,NaN,0.0,Transient,98.0,0.0,1.0,Check-Out,2015-07-03 00:00:00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182872,Resort Hotel,NaN,224.337762,NaN,NaN,NaN,NaN,19.005545,19.819823,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
182873,Resort Hotel,NaN,390.141963,NaN,NaN,NaN,NaN,14.751794,19.989726,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
182874,Resort Hotel,NaN,230.689826,NaN,NaN,NaN,NaN,11.409496,20.461372,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
182875,Resort Hotel,NaN,304.888534,NaN,NaN,NaN,NaN,16.744472,15.400773,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# CON ESTA FÓRMULA IDENTIFICAMOS LAS FILAS QUE TIENEN CASI TODOS LOS VALORES NULOS, POR SI QUEREMOS ELIMINARLAS


# Definir un umbral para considerar una fila como casi completamente nula (por ejemplo, 75% de nulos)
threshold = 0.75

# Calcular la proporción de valores nulos en cada fila
null_proportion = df.isna().mean(axis=1)

# Filtrar las filas que superan el umbral de nulos
rows_with_many_nans = df[null_proportion > threshold]

# Mostrar las filas con muchos valores nulos
print("Filas con muchos valores nulos:")
print(rows_with_many_nans)

# Mostrar las proporciones de nulos en esas filas
print("\nProporciones de nulos en esas filas:")
print(null_proportion[null_proportion > threshold])

Filas con muchos valores nulos:
       hotel is_canceled   lead_time  arrival_date_year arrival_date_month  \
119390   NaN         NaN         NaN                NaN                NaN   
119391   NaN         NaN         NaN                NaN                NaN   
119392   NaN         NaN         NaN                NaN                NaN   
119393   NaN         NaN         NaN                NaN                NaN   
119394   NaN         NaN         NaN                NaN                NaN   
...      ...         ...         ...                ...                ...   
182872   NaN         NaN  224.337762                NaN                NaN   
182873   NaN         NaN  390.141963                NaN                NaN   
182874   NaN         NaN  230.689826                NaN                NaN   
182875   NaN         NaN  304.888534                NaN                NaN   
182876   NaN         NaN  341.238166                NaN                NaN   

        arrival_date_week_numbe